In [3]:
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import numpy as np
import pandas as pd
import polyline

import sys
sys.path.append('..')
from api_key import api_key, mapbox_token

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

In [4]:
# Get data
payload = {
    'client_id': api_key['client_id'],
    'client_secret': api_key['client_secret'],
    'refresh_token': api_key['refresh_token'],
    'grant_type': "refresh_token",
    'f': 'json'
}

# Request access token
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']

# Get data
header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_dataset = requests.get(activites_url, headers=header, params=param).json()
df = pd.json_normalize(my_dataset)
df.head()

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,workout_type,id,external_id,...,pr_count,total_photo_count,has_kudoed,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state,average_temp,average_cadence
0,2,Rebild bakker,35085.0,10484,15388,537.4,Ride,10.0,5085121622,garmin_push_6565702584,...,2,0,False,623588,1,a5085121622,ue{yIguf{@@|IpApFjGiCtJa@nIbBjBmQjDyUy@jM~DkFu...,2,NaN,NaN
1,2,Aalborg,20125.5,6279,7416,435.2,Ride,10.0,5078818942,garmin_push_6559443035,...,9,0,False,623588,1,a5078818942,eg|zIetp{@p@_@jD`EbAk@McE{@gBBaB`@i@\Rd@pAr@YX...,2,NaN,NaN
2,2,Silkeborg-Viborg-Mors part 3,10816.7,3971,5083,179.0,Ride,10.0,5050831627,garmin_push_6532008192,...,0,0,False,623588,1,a5050831627,oslyI{ryt@@^ZRhBdCF~B[hEa@~BYv@w@bAMq@]\Oc@G@E...,2,NaN,NaN
3,2,Silkeborg-Viborg-Mors part 2,9026.9,2901,3238,69.3,Ride,10.0,5049722962,garmin_push_6530904449,...,0,0,False,623588,1,a5049722962,u~tvI_fsw@i@y@Dc@EW@iAOs@EaAXmA@SOg@MwBUuAy@sA...,2,NaN,NaN
4,2,Silkeborg-Viborg-Mors part 1,15201.3,4250,4899,319.3,Ride,10.0,5048541792,garmin_push_6529719534,...,0,0,False,623588,1,a5048541792,spuuIeliy@|ByB`CTN_@zC_DjAMBzARLZoCROTb@CvF`Ah...,2,NaN,NaN


In [7]:
# plot 1 ride
_df = df.head(1)

coordinates = polyline.decode(_df.loc[0,'map.summary_polyline'])

ride_longitudes = [coordinate[1] for coordinate in coordinates]
ride_latitudes = [coordinate[0] for coordinate in coordinates]

In [8]:
# plot ride above on map
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Scattermapbox(
        lat = ride_latitudes, 
        lon = ride_longitudes,
        mode = "lines", # is it possible to add direction arrow?
        line= go.scattermapbox.Line(
            color='black'
        )
    )
)

fig.update_layout(
        mapbox = go.layout.Mapbox(
            accesstoken=mapbox_token,
            style='light',
            zoom=11,
            center=go.layout.mapbox.Center(
                lat=np.median(ride_latitudes),
                lon=np.median(ride_longitudes)
            ),
        ),
        margin={"r":0,"t":0,"l":0,"b":0}
    )

In [9]:
import plotly.express as px

fig = px.line_mapbox(lat=ride_latitudes, lon=ride_longitudes, zoom=11,
                  height = 600, width = 1000, #center = dict(lat = g.center)
                        title='Drive Route with Mapbox',
                       #mapbox_style="open-street-map"
                       )
fig.update_layout(font_size=16,  title={'xanchor': 'center','yanchor': 'top', 'y':0.9, 'x':0.5,}, 
        title_font_size = 24, mapbox_accesstoken=mapbox_token, mapbox_style = "mapbox://styles/strym/ckhd00st61aum19noz9h8y8kw")
fig.update_traces(marker=dict(size=6))

In [65]:
df.head()

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,workout_type,id,external_id,...,pr_count,total_photo_count,has_kudoed,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state,average_temp,average_cadence
0,2,Rebild bakker,35085.0,10484,15388,537.4,Ride,10.0,5085121622,garmin_push_6565702584,...,2,0,False,623588,1,a5085121622,ue{yIguf{@@|IpApFjGiCtJa@nIbBjBmQjDyUy@jM~DkFu...,2,NaN,NaN
1,2,Aalborg,20125.5,6279,7416,435.2,Ride,10.0,5078818942,garmin_push_6559443035,...,9,0,False,623588,1,a5078818942,eg|zIetp{@p@_@jD`EbAk@McE{@gBBaB`@i@\Rd@pAr@YX...,2,NaN,NaN
2,2,Silkeborg-Viborg-Mors part 3,10816.7,3971,5083,179.0,Ride,10.0,5050831627,garmin_push_6532008192,...,0,0,False,623588,1,a5050831627,oslyI{ryt@@^ZRhBdCF~B[hEa@~BYv@w@bAMq@]\Oc@G@E...,2,NaN,NaN
3,2,Silkeborg-Viborg-Mors part 2,9026.9,2901,3238,69.3,Ride,10.0,5049722962,garmin_push_6530904449,...,0,0,False,623588,1,a5049722962,u~tvI_fsw@i@y@Dc@EW@iAOs@EaAXmA@SOg@MwBUuAy@sA...,2,NaN,NaN
4,2,Silkeborg-Viborg-Mors part 1,15201.3,4250,4899,319.3,Ride,10.0,5048541792,garmin_push_6529719534,...,0,0,False,623588,1,a5048541792,spuuIeliy@|ByB`CTN_@zC_DjAMBzARLZoCROTb@CvF`Ah...,2,NaN,NaN


In [60]:
for index, row in df.iterrows():
    pass

In [95]:
row

resource_state                                                                   2
name                                                                    06/15/2012
distance                                                                   15607.2
moving_time                                                                   2922
elapsed_time                                                                  3373
total_elevation_gain                                                         377.6
type                                                                          Ride
workout_type                                                                   NaN
id                                                                        10912156
external_id                                   B59D52EF-AACB-4ACC-ADEA-6601061A81E7
upload_id                                                                 12642242
start_date                                                    2012-06-16T01:04:15Z
star

In [105]:
df.iloc[109]

resource_state                                                 2
name                                                Morning Swim
distance                                                     0.0
moving_time                                                    0
elapsed_time                                                2281
total_elevation_gain                                         0.0
type                                                        Swim
workout_type                                                 NaN
id                                                    3883286122
external_id                               garmin_push_5354845609
upload_id                                             4155791370
start_date                                  2020-08-08T14:23:19Z
start_date_local                       2020-08-08 07:23:19+00:00
timezone                         (GMT-08:00) America/Los_Angeles
utc_offset                                              -25200.0
start_latlng             